In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [2]:
# Загрузка и предобработка датасета CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Определение модели CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)  # Разворачивание тензора
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [5]:
# Инициализация модели, функции потерь и оптимизатора
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Обучение модели
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/500], Loss: 1.2612
Epoch [1/5], Step [200/500], Loss: 1.1975
Epoch [1/5], Step [300/500], Loss: 1.0418
Epoch [1/5], Step [400/500], Loss: 0.8480
Epoch [1/5], Step [500/500], Loss: 0.9297
Epoch [2/5], Step [100/500], Loss: 0.9034
Epoch [2/5], Step [200/500], Loss: 0.8387
Epoch [2/5], Step [300/500], Loss: 0.8155
Epoch [2/5], Step [400/500], Loss: 0.6653
Epoch [2/5], Step [500/500], Loss: 0.9174
Epoch [3/5], Step [100/500], Loss: 0.6533
Epoch [3/5], Step [200/500], Loss: 0.6889
Epoch [3/5], Step [300/500], Loss: 0.7539
Epoch [3/5], Step [400/500], Loss: 0.8104
Epoch [3/5], Step [500/500], Loss: 0.8188
Epoch [4/5], Step [100/500], Loss: 0.5504
Epoch [4/5], Step [200/500], Loss: 0.5123
Epoch [4/5], Step [300/500], Loss: 0.6225
Epoch [4/5], Step [400/500], Loss: 0.4727
Epoch [4/5], Step [500/500], Loss: 0.6147
Epoch [5/5], Step [100/500], Loss: 0.4002
Epoch [5/5], Step [200/500], Loss: 0.5641
Epoch [5/5], Step [300/500], Loss: 0.4356
Epoch [5/5], Step [400/500], Loss:

In [7]:
# Оценка модели
model.eval()  # Переключение в режим оценки
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total}%')

Accuracy of the model on the 10000 test images: 74.18%
